In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

In [2]:
#Load training data
df = pd.read_csv('train.csv')
passengerId = df.iloc[:,0].to_numpy()
survived = df.iloc[:,1].to_numpy()
pclass = df.iloc[:,2].to_numpy() #important
name = df.iloc[:,3].to_numpy()
sex = df.iloc[:,4].to_numpy() #important
age = df.iloc[:,5].to_numpy() #important
sibsp = df.iloc[:,6].to_numpy() #important
parch = df.iloc[:,7].to_numpy() #important
ticket = df.iloc[:,8].to_numpy()
fare = df.iloc[:,9].to_numpy() #important
cabin = df.iloc[:,10].to_numpy()
embarked = df.iloc[:,11].to_numpy()

#Load test data
tdf = pd.read_csv('test.csv')
tpassengerId = tdf.iloc[:,0].to_numpy()
tpclass = tdf.iloc[:,1].to_numpy()
tname = tdf.iloc[:,2].to_numpy()
tsex = tdf.iloc[:,3].to_numpy()
tage = tdf.iloc[:,4].to_numpy()
tsibsp = tdf.iloc[:,5].to_numpy()
tparch = tdf.iloc[:,6].to_numpy()
tticket = tdf.iloc[:,7].to_numpy()
tfare = tdf.iloc[:,8].to_numpy()
tcabin = tdf.iloc[:,9].to_numpy()
tembarked = tdf.iloc[:,10].to_numpy()

print(passengerId[5])
print(embarked[5])


6
Q


In [22]:
#data processing
for i in range(len(sex)):
    if(sex[i] == "male"):
        sex[i] = 1
    else:
        sex[i] = 2
        
for i in range(len(tsex)):
    if(tsex[i] == "male"):
        tsex[i] = 1
    else:
        tsex[i] = 2

print(sex[0])

for i in range(len(age)):
    if(np.isnan(age[i])):
        age[i] = -1

for i in range(len(tage)):
    if(np.isnan(tage[i])):
        tage[i] = -1

for i in range(len(tfare)):
    if(np.isnan(tfare[i])):
        tfare[i] = -1

2


In [4]:
class net(torch.nn.Module):
    def __init__(self, input_size):
        super(net, self).__init__()
        size1 = 128
        size2 = 32
        size3 = 8
        size4 = 4
        
        self.fc1 = nn.Linear(input_size, size1)
        self.fc2 = nn.Linear(size1, size2)
        self.fc3 = nn.Linear(size2, size3)
        self.fc4 = nn.Linear(size3, size4)
        self.fc5 = nn.Linear(size4, 1)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        return torch.sigmoid(self.fc5(x))

In [5]:
model = net(6)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.00001)
loss_fn = torch.nn.BCELoss()

In [6]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, data, targets):
        self.data = torch.from_numpy(data)
        self.targets = torch.from_numpy(targets)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

In [7]:
from torch.utils.data import DataLoader, random_split
sex = sex.astype(int)
data = np.stack((pclass, sex, age, sibsp, parch, fare), axis = -1)
target = survived
dataset = MyDataset(data, target)
train_len = int(len(dataset) * 0.8)
val_len = len(dataset) - train_len

train_data, val_data = random_split(dataset, [train_len, val_len])
print(len(train_data))
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=16, shuffle=True)

for d in train_loader:
    print(d[0].shape)
    print(d[1].shape)
    break

712
torch.Size([16, 6])
torch.Size([16])


In [8]:
print(train_data[2][0].float())
output = model(train_data[2][0].float())
output = output
print(output)

tensor([ 3.0000,  1.0000, 25.0000,  0.0000,  0.0000,  7.0500])
tensor([0.5788], grad_fn=<SigmoidBackward0>)


In [9]:
for epoch in range(500):
    for i, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        
        output = model(data.float())
        output = output.squeeze()
        loss = loss_fn(output, target.float())
        #print(output.round())
        #print(target)
        loss.backward()
        optimizer.step()
        #print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, i * len(data), len(train_loader.dataset), 100. * i / len(train_loader), loss.item()))
        #print('Accuracy: {:.6f}'.format((output.round() == target).sum().item() / len(target)))
    val_loss = 0
    val_accuracy = 0
    
    for i, (data, target) in enumerate(val_loader):
        output = model(data.float())
        output = output.squeeze()
        loss = loss_fn(output, target.float())
        val_loss += loss.item()
        val_accuracy = (val_accuracy*i+(output.round() == target).sum().item() / len(target))/(i+1)
    print('Val Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, i * len(data), len(val_loader.dataset), 100. * i / len(val_loader), val_loss))
    print('Accuracy: {:.6f}'.format(val_accuracy))

Val Epoch: 0 [33/179 (92%)]	Loss: 8.840058
Accuracy: 0.359375
Val Epoch: 1 [33/179 (92%)]	Loss: 8.621694
Accuracy: 0.404514


Val Epoch: 2 [33/179 (92%)]	Loss: 8.721044
Accuracy: 0.381944
Val Epoch: 3 [33/179 (92%)]	Loss: 8.634454
Accuracy: 0.381944
Val Epoch: 4 [33/179 (92%)]	Loss: 8.689341
Accuracy: 0.381944
Val Epoch: 5 [33/179 (92%)]	Loss: 8.681418
Accuracy: 0.359375
Val Epoch: 6 [33/179 (92%)]	Loss: 8.518816
Accuracy: 0.404514
Val Epoch: 7 [33/179 (92%)]	Loss: 8.571443
Accuracy: 0.381944
Val Epoch: 8 [33/179 (92%)]	Loss: 8.643793
Accuracy: 0.359375
Val Epoch: 9 [33/179 (92%)]	Loss: 8.496601
Accuracy: 0.404514
Val Epoch: 10 [33/179 (92%)]	Loss: 8.573595
Accuracy: 0.381944
Val Epoch: 11 [33/179 (92%)]	Loss: 8.513764
Accuracy: 0.404514
Val Epoch: 12 [33/179 (92%)]	Loss: 8.441739
Accuracy: 0.404514
Val Epoch: 13 [33/179 (92%)]	Loss: 8.569938
Accuracy: 0.381944
Val Epoch: 14 [33/179 (92%)]	Loss: 8.587345
Accuracy: 0.359375
Val Epoch: 15 [33/179 (92%)]	Loss: 8.615381
Accuracy: 0.359375
Val Epoch: 16 [33/179 (92%)]	Loss: 8.526027
Accuracy: 0.381944
Val Epoch: 17 [33/179 (92%)]	Loss: 8.453268
Accuracy: 0.404514


In [34]:
tsex = tsex.astype(int)
testdata = np.stack((tpclass, tsex, tage, tsibsp, tparch, tfare), axis = -1)
arr1 = np.zeros((len(testdata), 1))
testdataset = MyDataset(testdata, arr1)
testloader = DataLoader(testdataset, batch_size = 1, shuffle = False)

df = pd.read_csv('gender_submission.csv')

for i, (data,target) in enumerate(testloader):
    output = model(data.float())
    num = int(output.round().item())
    print(num)
    df.iloc[i, 1] = num
    df.iloc[i, 1] = df.iloc[i, 1].astype(int)
for col in df.columns:
    df[col] = df[col].astype(int)
df.to_csv('gender_submission.csv', index = False)

1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
1
1
1
1
1
1
0
1
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
0
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
1
1
1
1
0
0
1
1
1
1
0
1
1
1
1
1
1
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
0
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
